In [474]:
# 1.	Importing the standard libraries

import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.linear_model
from sklearn.model_selection import KFold

In [475]:
# 2.	Importing the dataset
#       a.	Using sklearn

bostonData = load_boston()

In [476]:
# 3.	Viewing the dataset
#     a.	“the number of features”
#     b.	“the features names:”
#     c.	“the number of examples”

#  create X and Y variables - X holding the .data and Y holding .target 
x = bostonData.data
y = bostonData.target

numFeatures = x.shape[1]
numPoints = x.shape[0]
featureNames = bostonData.feature_names
firstTwoRows = x[0:2]

print("Number of features:",numFeatures)
print("Number of datapoints:",numPoints)
print("Name of features:",featureNames)
print("First Two Rows:",firstTwoRows)

Number of features: 13
Number of datapoints: 506
Name of features: ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
First Two Rows: [[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]]


In [477]:
# 4.	Transfer to polynomial feature
#     a.	Using API
#     b.	Print the shape of new freature 

def transformPolyFeature(x):
    poly = PolynomialFeatures(degree=2)

    return  poly.fit_transform(x)


In [478]:
# 5.	Implementing the function that return the close form w for linear regression 
#     a.	Return w values

# 6.	Implementing the function that return the close form w for ridge regression 
#     a.	Return w values

def linearRidgeRegressionClosedForm(xTrainTemp,yTrain, lambdaVar):
    if len(yTrain.shape) == 1: 
        yTrain = yTrain[:, np.newaxis]
        
    A = lambdaVar * np.identity(xTrainTemp.shape[1]) 
    B = np.linalg.pinv(np.matmul(np.transpose(xTrainTemp),xTrainTemp)+A)
    
    return np.matmul(np.matmul(B,np.transpose(xTrainTemp)),yTrain)


In [479]:
# 7.	Implementing the evaluation function
#     a.	Return the trainError and testError values

def errorEvaluation(xTrainTemp, xTestTemp, yTrain, yTest, wCoeffecients):
   
    predTrain = (np.matmul(xTrainTemp,wCoeffecients).squeeze()) * np.std(yTrain) + np.mean(yTrain)

    predTest = (np.matmul(xTestTemp,wCoeffecients).squeeze()) * np.std(yTrain) + np.mean(yTrain)
    
    tempTrain = (yTrain-predTrain)**2
    tempTest = (yTest-predTest)**2
    
    sizeTrain = yTrain.shape[0]
    sizeTest = yTest.shape[0]
    
    trainError = np.sum(tempTrain) / sizeTrain
    testError = np.sum(tempTest) / sizeTest
    return trainError, testError



In [480]:
# 8.	Finish writting the k_fold_cross_validation function. 
#     a.	Returns the average training error and average test error from the k-fold cross validation
#     b.	use Sklearns K-Folds cross-validator: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
#     c.	centering the data so we do not need the intercept term (we could have also chose w_0=average y value)
#     d.	scaling the data matrix
#     e.	determine the training error and the test error

def k_fold_cross_validation(k,x,y,lambdaVar):
    
    kFold = KFold(n_splits = k, shuffle = True)
    trainingErrorValues = []
    testingErrorValues = []
    
    for trainingDataIndex, testingDataIndex in kFold.split(x):
  
        xTrain = x[trainingDataIndex]
        xTest = x[testingDataIndex]
        
        yTrain, yTest = y[trainingDataIndex], y[testingDataIndex]

        scaleVal = preprocessing.StandardScaler().fit(xTrain)
        xTrain = scaleVal.transform(xTrain)
        xTest = scaleVal.transform(xTest)

        xTrainMean = np.mean(xTrain)
        xTrainStd = np.std(xTrain)
        xTrainStd = np.where(xTrainStd == 0, 1, xTrainStd)
        
        xTrain = (xTrain - xTrainMean)/xTrainStd
        xTrainTemp = np.ones((xTrain.shape[0], x.shape[1] + 1))
        xTrainTemp[:, :x.shape[1]] = xTrain
        
        xTest = (xTest - xTrainMean)/xTrainStd
        xTestTemp = np.ones((xTest.shape[0], x.shape[1] + 1))
        xTestTemp[:,:x.shape[1]] = xTest
        
        
        yTrainMean = np.mean(yTrain)
        yTrainStd = np.std(yTrain)
        yTrainTemp = (yTrain - yTrainMean)/yTrainStd
        
        wCoeffecients = linearRidgeRegressionClosedForm(xTrainTemp,yTrainTemp,lambdaVar)
        
        
        trainingErrorValueTemp, testingErrorValueTemp = errorEvaluation(xTrainTemp, xTestTemp, yTrain, yTest, wCoeffecients)
   


        trainingErrorValues.append(trainingErrorValueTemp)
        testingErrorValues.append(testingErrorValueTemp)

    return trainingErrorValues, testingErrorValues, wCoeffecients

 

In [481]:
# print the error for the both linear regression and ridge regression
# the error should include both training error and testing error

trainingErrorValuesLinear, testingErrorValuesLinear, wCoeffecients = k_fold_cross_validation(10,x,y,0)

print("Linear Training Error",sum(trainingErrorValuesLinear)/len(trainingErrorValuesLinear))
print("Linear Testing Error",sum(testingErrorValuesLinear)/len(testingErrorValuesLinear))

lambdaValRange = np.logspace(1, 7, num=13)
finLambdaVar = 0
minTestError = float("inf")
minTrainError = float("inf")
for i in lambdaValRange:
    trainingErrorValuesRidge, testingErrorValuesRidge, wCoeffecients = k_fold_cross_validation(10,x,y,i)
    averageTestingError = np.sum(testingErrorValuesRidge)/10
    averageTrainingError = np.sum(trainingErrorValuesRidge)/10
    if minTestError > averageTestingError:
        minTestError = averageTestingError
        minTrainError = averageTrainingError
        finLambdaVar = i
print("\nBest Lambda:", finLambdaVar)
print("\nRidge Training Error",minTrainError)
print("Ridge Testing Error",minTestError)



Linear Training Error 21.794178077854117
Linear Testing Error 23.877623713602993

Best Lambda: 10.0

Ridge Training Error 21.898427214241682
Ridge Testing Error 23.503067330575597


In [482]:
xTransformed = transformPolyFeature(x)
xTransformed = xTransformed[:,1:]

trainingErrorValuesLinear, testingErrorValuesLinear, wCoeffecients = k_fold_cross_validation(10,xTransformed,y,0)

print("Linear Training Error",sum(trainingErrorValuesLinear)/len(trainingErrorValuesLinear))
print("Linear Testing Error",sum(testingErrorValuesLinear)/len(testingErrorValuesLinear))

lambdaValRange = np.logspace(1, 7, num=13)
finLambdaVar = 0
minTestError = float("inf")
minTrainError = float("inf")
finWCoeffecients = 0
for i in lambdaValRange:
    trainingErrorValuesRidge, testingErrorValuesRidge, wCoeffecients = k_fold_cross_validation(10,xTransformed,y,i)
    averageTestingError = np.sum(testingErrorValuesRidge)/10
    averageTrainingError = np.sum(trainingErrorValuesRidge)/10
    if minTestError > averageTestingError:
        minTestError = averageTestingError
        minTrainError = averageTrainingError
        finLambdaVar = i
        finWCoeffecients = wCoeffecients
print("\nBest Lambda:", finLambdaVar)
print("\nRidge Training Error",minTrainError)
print("Ridge Testing Error",minTestError)

Linear Training Error 5.758004343418935
Linear Testing Error 13.703209742296433

Best Lambda: 10.0

Ridge Training Error 10.04027864980662
Ridge Testing Error 13.770060964084388


In [488]:
# I would select a closed form ridge regression model with a second degree
# polynomial transormation. This is because it produces the lowest average
# testing error. The paramters for this model are as follows: 

print("Model Parameters - K:",10)
print("\nModel Parameters - Lambda:",finLambdaVar)
print("\nModel Parameters - W:\n",finWCoeffecients.squeeze())


Model Parameters - K: 10

Model Parameters - Lambda: 10.0

Model Parameters - W:
 [ 2.08304526e-02 -5.26832327e-02  6.94457848e-02  6.14940224e-02
  3.21368118e-02  2.02638096e-01  7.15256890e-02 -1.37354001e-01
  1.49821336e-01  3.96991886e-02 -6.93921182e-03  2.35728817e-02
  1.02910451e-02  8.12909395e-02  4.19910867e-02  2.46448160e-02
  2.41958481e-01 -6.13435305e-02 -1.11864257e-01  1.69899071e-02
 -7.50439693e-02 -1.21787032e-02  4.78814376e-03  1.16759402e-02
 -6.00477515e-02  4.57134224e-03  6.45987168e-02 -6.93777691e-02
 -1.01164018e-02 -2.68089877e-02  7.93211237e-02  3.54475548e-03
 -2.35992880e-03 -2.06110893e-03  6.00961939e-02  5.42073390e-02
 -4.29594544e-02 -7.15508440e-02  6.13567385e-02 -2.06079330e-02
  4.66078611e-02 -1.02294060e-01  6.70302680e-02 -7.73383887e-02
  8.75537291e-02  1.01014030e-01 -3.54060578e-02  7.50967595e-02
 -1.42605935e-01  6.14940224e-02 -1.98640973e-01 -1.27416381e-01
  9.42868915e-02  4.57460857e-02 -1.11258642e-01 -6.94199628e-02
  7.0833